<a href="https://colab.research.google.com/github/velardep/Red_Neuronal-_de-_Clasificaci-n-_de-_Contenidos-_Pedag-gicos-_-LSTM-/blob/main/RED_1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, os
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("DEVICE:", "cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    torch.backends.cudnn.benchmark = True


torch: 2.9.0+cu126
cuda available: True
DEVICE: cuda
GPU: Tesla T4


In [2]:
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install transformers scikit-learn pandas numpy tqdm pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 12.6 MB/s eta 0:00:00


In [3]:
import os, re, json, random, numpy as np, pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from pypdf import PdfReader
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, classification_report

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE, "| torch:", torch.__version__)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    torch.backends.cudnn.benchmark = True


DEVICE: cuda | torch: 2.9.0+cu126
GPU: Tesla T4


In [5]:
# ✅ CELDA 3: Carga PDFs (Secundaria + Primaria) y extrae texto (PDFs no escaneados)

from pypdf import PdfReader

# Nombres exactos de los archivos que subiste
PDF_SEC = "Prontuario de mis aprendizajes - Educación Secundaria Comunitaria Productiva.pdf"
PDF_PRI = "Prontuario de mis aprendizajes - Educación Primaria Comunitaria Vocacional.pdf"

# Función para buscar PDF en rutas típicas
def find_pdf(pdf_name):
    candidate_paths = [
        f"/content/{pdf_name}",
        f"/mnt/data/{pdf_name}",
        pdf_name,  # por si está en el cwd
    ]
    for p in candidate_paths:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(
        f"No encontré el PDF. Sube el archivo con el mismo nombre: {pdf_name} "
        f"y verifica que esté en /content. Paths intentados: {candidate_paths}"
    )

# Buscar y leer PDFs
sec_path = find_pdf(PDF_SEC)
pri_path = find_pdf(PDF_PRI)

print("PDF Secundaria encontrado en:", sec_path)
print("PDF Primaria encontrado en:", pri_path)

# Función para extraer texto de un PDF
def extract_text_from_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += (page.extract_text() or "") + "\n"
    return text

# Extraer textos
full_text_sec = extract_text_from_pdf(sec_path)
full_text_pri = extract_text_from_pdf(pri_path)

# Combinar textos
full_text = full_text_sec + "\n" + full_text_pri

print("Caracteres totales combinados extraídos:", len(full_text))


PDF Secundaria encontrado en: /content/Prontuario de mis aprendizajes - Educación Secundaria Comunitaria Productiva.pdf
PDF Primaria encontrado en: /content/Prontuario de mis aprendizajes - Educación Primaria Comunitaria Vocacional.pdf
Caracteres totales combinados extraídos: 2430066


In [6]:
# ✅ CELDA 4: Limpieza y división en chunks robustos (para PDFs largos)
def clean_text(text: str) -> str:
    text = text.replace("\x00", " ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def chunk_text(text: str, max_words: int = 260, overlap_words: int = 60):
    # Divide por palabras con overlap para no perder contexto en límites
    words = text.split()
    if len(words) < 30:
        return []

    chunks = []
    start = 0
    while start < len(words):
        end = min(start + max_words, len(words))
        c = " ".join(words[start:end]).strip()
        if len(c) >= 120:  # filtro mínimo para evitar basura
            chunks.append(c)
        if end == len(words):
            break
        start = max(0, end - overlap_words)

    return chunks

full_text = clean_text(full_text)
chunks = chunk_text(full_text, max_words=260, overlap_words=60)

print("Total de chunks:", len(chunks))
print("Ejemplo de chunk (primeros 350 chars):\n", chunks[0][:350])


Total de chunks: 1874
Ejemplo de chunk (primeros 350 chars):
 Prontuario de mis aprendizajes Educación Secundaria Comunitaria Productiva Omar Veliz Ramos MINISTRO DE EDUCACIÓN Manuel Eudal Tejerina del Castillo VICEMINISTRO DE EDUCACIÓN REGULAR Delia Yucra Rodas DIRECTORA GENERAL DE EDUCACIÓN SECUNDARIA Equipo de redacción Dirección General de Educación Secundaria Revisión Instituto de Investigaciones Pedagóg


In [7]:
# ✅ CELDA 5: Define etiquetas finales (ÁREA x DIMENSIÓN)
# Reglas:
# - Áreas/Componentes típicos en secundaria (ajustable luego, pero estas quedan fijas en el modelo)
# - Dimensiones: SABER / HACER / SER / DECIDIR
# - Etiqueta final = "{AREA}__{DIMENSION}"  -> multi-label

AREAS = [
    "MATEMATICA",
    "COMUNICACION_Y_LENGUAJES",
    "CIENCIAS_NATURALES_BIOLOGIA",
    "FISICA",
    "QUIMICA",
    "CIENCIAS_SOCIALES",
    "VALORES_ESPIRITUALIDAD_RELIGIONES",
    "TECNOLOGIA_PRODUCTIVA",
]

DIMENSIONES = ["SABER", "HACER", "SER", "DECIDIR"]

LABELS = [f"{a}__{d}" for a in AREAS for d in DIMENSIONES]

print("Total de etiquetas:", len(LABELS))
print("Primeras 12 etiquetas:", LABELS[:12])
print("Últimas 12 etiquetas:", LABELS[-12:])


Total de etiquetas: 32
Primeras 12 etiquetas: ['MATEMATICA__SABER', 'MATEMATICA__HACER', 'MATEMATICA__SER', 'MATEMATICA__DECIDIR', 'COMUNICACION_Y_LENGUAJES__SABER', 'COMUNICACION_Y_LENGUAJES__HACER', 'COMUNICACION_Y_LENGUAJES__SER', 'COMUNICACION_Y_LENGUAJES__DECIDIR', 'CIENCIAS_NATURALES_BIOLOGIA__SABER', 'CIENCIAS_NATURALES_BIOLOGIA__HACER', 'CIENCIAS_NATURALES_BIOLOGIA__SER', 'CIENCIAS_NATURALES_BIOLOGIA__DECIDIR']
Últimas 12 etiquetas: ['CIENCIAS_SOCIALES__SABER', 'CIENCIAS_SOCIALES__HACER', 'CIENCIAS_SOCIALES__SER', 'CIENCIAS_SOCIALES__DECIDIR', 'VALORES_ESPIRITUALIDAD_RELIGIONES__SABER', 'VALORES_ESPIRITUALIDAD_RELIGIONES__HACER', 'VALORES_ESPIRITUALIDAD_RELIGIONES__SER', 'VALORES_ESPIRITUALIDAD_RELIGIONES__DECIDIR', 'TECNOLOGIA_PRODUCTIVA__SABER', 'TECNOLOGIA_PRODUCTIVA__HACER', 'TECNOLOGIA_PRODUCTIVA__SER', 'TECNOLOGIA_PRODUCTIVA__DECIDIR']


In [13]:
# ✅ CELDA 6: Auto-etiquetado determinista (reglas) para poder ENTRENAR hoy con tu PDF
# IMPORTANTE:
# - Esto NO es "simulación aleatoria".
# - Es weak-supervision: reglas reproducibles basadas en palabras-clave.
# - En producción, puedes mejorar etiquetado con validación docente (active learning) sin cambiar arquitectura.

AREA_KEYWORDS = {
    "MATEMATICA": [
        # Conceptos generales y Aritmética
        "números y operaciones", "sistemas de numeración", "conjuntos numéricos",
        "números naturales", "números racionales", "números reales",
        "números irracionales", "números primos", "números decimales",
        "número entero", "numeración binaria", "razones", "proporciones",
        "regla de tres", "análisis numérico", "lógica matemática",
        # Álgebra y Cálculo
        "álgebra", "expresiones algebraicas", "ecuaciones de primer grado",
        "ecuaciones cuadráticas", "sistemas de ecuaciones", "desigualdades",
        "inecuaciones", "factorización", "fracciones algebraicas",
        "exponentes", "radicales", "números complejos", "sucesiones",
        "progresiones", "análisis matemático", "cálculo infinitesimal",
        "límites", "continuidad", "derivada", "integrales", "álgebra lineal",
        # Geometría y Trigonometría
        "geometría", "plano cartesiano", "triángulos", "semejanza",
        "áreas", "perímetros", "formas tridimensionales", "geometría analítica",
        "línea recta", "circunferencia", "parábola", "elipse", "hipérbola",
        "trigonometría", "identidades trigonométricas", "análisis vectorial",
        # Estadística y Otros
        "estadística", "probabilidad", "combinatoria", "matemática financiera",
        "teoría de conjuntos", "automatización", "inteligencia artificial"
    ],
    "COMUNICACION_Y_LENGUAJES": [
        # Lingüística y Gramática
        "lenguaje oral", "lenguaje escrito", "signos sonoros", "fonología",
        "fonética", "habla", "morfología gramatical", "verbos regulares",
        "sustantivos", "adjetivos", "neologismos", "palabras homógrafas",
        "palabras multiformes", "polisémicas", "definiciones", "mayúsculas",
        # Tipos de texto y Comunicación
        "medios de comunicación social", "masas", "géneros periodísticos",
        "textos orales", "cómic", "comunicación efectiva", "monografía",
        "investigación", "referencias bibliográficas", "justificación",
        "objetivos", "mapas mentales", "mapa conceptual", "trabalenguas"
    ],
    "CIENCIAS_NATURALES_BIOLOGIA": [
        # Ramas de la Biología
        "biología", "botánica", "zoología", "ecología", "microbiología",
        "virología", "genética", "entomología", "ornitología", "herpetología",
        "bacteriología", "protozoología", "pteridología", "ficología",
        "ictiología", "biomatemáticas", "biogeografía", "bioquímica",
        "biofísica", "bioética",
        # Conceptos Biológicos
        "vida", "organismo", "célula", "moléculas orgánicas", "nutrientes",
        "procesos metabólicos", "evolución", "crecimiento", "origen",
        "distribución", "ecosistemas", "naturaleza"
    ],
    "FISICA": [
        # Ramas de la Física
        "física clásica", "física moderna", "física contemporánea", "mecánica",
        "termodinámica", "movimiento ondulatorio", "óptica", "electromagnetismo",
        "relatividad", "mecánica cuántica", "física de partículas", "gravitación",
        "dinámica no lineal", "sistemas complejos", "nanofísica",
        # Conceptos y Mediciones
        "materia", "energía", "fenómenos naturales", "mediciones", "magnitudes",
        "cantidad patrón", "cinemática", "dinámica", "velocidad", "tiempo",
        "espacio", "vector", "campo vectorial", "fuerza"
    ],
    "QUIMICA": [
        # General y Ramas
        "química general", "estructura de la materia", "composición",
        "transformaciones", "elementos naturales",
        # Aplicaciones y Contexto
        "fármacos", "plaguicidas", "armas químicas", "compuestos",
        "análisis empírico", "contexto inorgánico"
    ],
    "CIENCIAS_SOCIALES": [
        # Disciplinas
        "historia", "antropología", "sociología", "ciencia política",
        "economía", "lingüística", "psicología", "derecho", "geografía",
        "demografía", "arqueología", "ciencias de la educación",
        # Conceptos Políticos y Sociales
        "estado plurinacional", "constitución política", "democracia",
        "gobierno", "poder político", "conflictos sociales", "instituciones",
        "territorialidad", "función pública", "ley 450", "vulnerabilidad",
        "naciones y pueblos indígena originarios", "aislamiento voluntario",
        # Economía
        "producción", "distribución", "consumo", "bienes y servicios",
        "producto interno bruto (pib)", "desarrollo económico", "comercio internacional"
    ],
    "VALORES_ESPIRITUALIDAD_RELIGIONES": [
        # Principios Andinos y Constitucionales
        "vivir bien", "suma qamaña", "ñandereko", "teko kavi", "ivi maraei",
        "qhapaj ñan", "ama qhilla", "ama llulla", "ama suwa",
        # Valores Éticos
        "unidad", "igualdad", "inclusión", "dignidad", "libertad",
        "solidaridad", "reciprocidad", "respeto", "complementariedad",
        "armonía", "transparencia", "equilibrio", "bienestar común",
        "responsabilidad", "justicia social", "vida armoniosa"
    ],

    "TECNOLOGIA_PRODUCTIVA": [
        "inteligencia artificial", "programación", "automatización", "robot",
        "redes neuronales", "chatbot", "proyecto", "plan sectorial",
        "necesidad", "solución de problemas", "intervención", "investigación"
    ],
}

DIM_KEYWORDS = {
    "SABER": [
        "definir", "reconocer", "identificar", "conocer", "describir",
        "explicar", "comprender", "interpretar", "analizar", "estudiar",
        "adquirir conocimientos", "profundizar", "entender", "teorizar"
    ],
    "HACER": [
        "resolver", "aplicar", "elaborar", "construir", "desarrollar",
        "realizar", "calcular", "producir", "experimentar", "investigar",
        "redactar", "medir", "comparar", "transformar", "ejecutar",
        "diseñar", "practicar", "utilizar"
    ],
    "SER": [
        "valorar", "respetar", "asumir", "convivir", "colaborar",
        "responsable", "solidario", "ético", "actitud", "reciprocidad",
        "complementariedad", "integridad", "consciente", "dignidad",
        "inclusión", "equidad"
    ],
    "DECIDIR": [
        "decidir", "elegir", "proponer", "planificar", "argumentar",
        "evaluar", "seleccionar", "priorizar", "comprometer", "promover",
        "transformar la realidad", "satisfacer necesidades", "vincular",
        "integrar", "participar", "solucionar"
    ],
}

MIN_AREA_HITS = 2      # exige evidencia para asignar área
MAX_DIMS = 2           # evita explotar etiquetas
FALLBACK = "CIENCIAS_SOCIALES__SABER"

def label_area(text: str):
    areas = infer_areas(text)          # tu función
    return areas[0] if areas else None

def label_dims(text: str):
    dims = infer_dims(text)            # tu función
    return dims if dims else None

rows = []
for c in tqdm(chunks, desc="Etiquetando chunks"):
    a = label_area(c)
    d = label_dims(c)
    if a is None or d is None:
        continue  # fuera basura
    rows.append({"text": c, "area": a, "dims": d})

df = pd.DataFrame(rows).reset_index(drop=True)
print(df.shape)
print(df.head())



Etiquetando chunks: 100%|██████████| 1874/1874 [00:01<00:00, 1127.14it/s]


(995, 3)
                                                text  \
0  Prontuario de mis aprendizajes Educación Secun...   
1  general DE MIS APRENDIZAJES Geometría analític...   
2  Óxidos ácidos o anhídridos ......................   
3  583 Articulaciones ..............................   
4  lenguajes Comunicación La comunicación ..........   

                          area              dims  
0                   MATEMATICA           [SABER]  
1                   MATEMATICA           [SABER]  
2  CIENCIAS_NATURALES_BIOLOGIA           [SABER]  
3     COMUNICACION_Y_LENGUAJES           [SABER]  
4     COMUNICACION_Y_LENGUAJES  [HACER, DECIDIR]  


In [14]:
# Celda 7

TOKENIZER_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

MAX_LEN = 256
BATCH_SIZE = 32

# Área (single label)
area_to_id = {a:i for i,a in enumerate(AREAS)}
y_area = np.array([area_to_id[a] for a in df["area"].tolist()])

# Dimensiones (multi-label 4)
mlb_dim = MultiLabelBinarizer(classes=DIMENSIONES)
y_dim = mlb_dim.fit_transform(df["dims"].values)

X_train, X_val, ya_train, ya_val, yd_train, yd_val = train_test_split(
    df["text"].tolist(), y_area, y_dim, test_size=0.2, random_state=SEED
)

enc_train = tokenizer(X_train, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
enc_val   = tokenizer(X_val,   truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")

class DualDataset(Dataset):
    def __init__(self, enc, ya, yd):
        self.ids = enc["input_ids"]
        self.att = enc["attention_mask"]
        self.ya = torch.tensor(ya).long()
        self.yd = torch.tensor(yd).float()

    def __len__(self): return self.ids.size(0)

    def __getitem__(self, idx):
        return {"input_ids": self.ids[idx], "attention_mask": self.att[idx], "y_area": self.ya[idx], "y_dim": self.yd[idx]}

train_ds = DualDataset(enc_train, ya_train, yd_train)
val_ds   = DualDataset(enc_val,   ya_val,   yd_val)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

print("Train:", len(train_ds), "Val:", len(val_ds))



Train: 796 Val: 199


In [26]:
# ✅ CELDA 8: Modelo LSTM multi-head + entrenamiento robusto (SER/DECIDIR raras)

import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

VOCAB_SIZE = tokenizer.vocab_size
PAD_ID = tokenizer.pad_token_id
NUM_AREAS = len(AREAS)
NUM_DIMS  = len(DIMENSIONES)

EMB_DIM = 256
HIDDEN = 256

class MultiHeadPedagogicalLSTM(nn.Module):
    def __init__(self, vocab_size, pad_id, emb_dim, hidden_dim, num_areas, num_dims):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.3)

        self.fc_area = nn.Linear(hidden_dim * 2, num_areas)  # softmax
        self.fc_dim  = nn.Linear(hidden_dim * 2, num_dims)   # sigmoid multi-label

    def forward(self, input_ids, attention_mask):
        x = self.embedding(input_ids)      # [B,T,E]
        out, _ = self.lstm(x)              # [B,T,2H]

        mask = attention_mask.unsqueeze(-1).float()  # [B,T,1]
        out = out * mask
        pooled = out.sum(dim=1) / mask.sum(dim=1).clamp(min=1e-6)  # [B,2H]

        pooled = self.dropout(pooled)

        logits_area = self.fc_area(pooled)  # [B,num_areas]
        logits_dim  = self.fc_dim(pooled)   # [B,num_dims]
        return logits_area, logits_dim

model = MultiHeadPedagogicalLSTM(VOCAB_SIZE, PAD_ID, EMB_DIM, HIDDEN, NUM_AREAS, NUM_DIMS).to(DEVICE)

# ✅ PESOS para ÁREA (desbalance)
area_counts = np.bincount(ya_train, minlength=NUM_AREAS).astype(np.float32)
area_weights = (area_counts.sum() / (area_counts + 1e-6))
area_weights = torch.tensor(area_weights, dtype=torch.float32).to(DEVICE)
loss_area = nn.CrossEntropyLoss(weight=area_weights)

# ✅ pos_weight + Focal para DIM (SER/DECIDIR raras)
pos = yd_train.sum(axis=0)                 # positivos por dim
neg = yd_train.shape[0] - pos
pos_weight_dim = torch.tensor(neg / (pos + 1e-6), dtype=torch.float32).to(DEVICE)

class FocalBCEWithLogitsLoss(nn.Module):
    def __init__(self, gamma=2.0, pos_weight=None):
        super().__init__()
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction="none")

    def forward(self, logits, targets):
        bce = self.bce(logits, targets)  # [B,num_dims]
        p = torch.sigmoid(logits)
        p_t = p * targets + (1 - p) * (1 - targets)
        focal = (1 - p_t).pow(self.gamma)
        return (focal * bce).mean()

loss_dim = FocalBCEWithLogitsLoss(gamma=2.0, pos_weight=pos_weight_dim)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=0.01)

EPOCHS = 20
print("Entrenando multi-head...")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total = 0.0

    for b in tqdm(train_dl, desc=f"Epoch {epoch}/{EPOCHS}"):
        ids = b["input_ids"].to(DEVICE)
        att = b["attention_mask"].to(DEVICE)
        y_a = b["y_area"].to(DEVICE)         # [B]
        y_d = b["y_dim"].to(DEVICE)          # [B,4]

        optimizer.zero_grad()
        logits_a, logits_d = model(ids, att)

        la = loss_area(logits_a, y_a)
        ld = loss_dim(logits_d, y_d)

        loss = la + 1.0 * ld  # ✅ peso 1:1 estable (no invento nada)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total += loss.item()

    print(f"Epoch {epoch} | Loss: {total/len(train_dl):.4f}")


Entrenando multi-head...


Epoch 1/20: 100%|██████████| 25/25 [00:00<00:00, 25.66it/s]


Epoch 1 | Loss: 2.3214


Epoch 2/20: 100%|██████████| 25/25 [00:00<00:00, 26.52it/s]


Epoch 2 | Loss: 2.2799


...

Epoch 18 | Loss: 0.9559


Epoch 19/20: 100%|██████████| 25/25 [00:00<00:00, 28.35it/s]


Epoch 19 | Loss: 0.9167


Epoch 20/20: 100%|██████████| 25/25 [00:00<00:00, 25.77it/s]

Epoch 20 | Loss: 0.8602


In [27]:
# ✅ CELDA 9: Calibración threshold para DIM (solo SABER/HACER) + export calibration.json

import numpy as np
import json
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, classification_report

def eval_dim_threshold(th, top_k=2):
    model.eval()
    all_probs, all_true = [], []

    with torch.no_grad():
        for b in val_dl:
            ids = b["input_ids"].to(DEVICE, non_blocking=True)
            att = b["attention_mask"].to(DEVICE, non_blocking=True)
            y_true = b["y_dim"].cpu().numpy()  # [N,4]

            _, logits_d = model(ids, att)
            probs_d = torch.sigmoid(logits_d).cpu().numpy()  # [N,4]

            all_probs.append(probs_d)
            all_true.append(y_true)

    y_prob = np.vstack(all_probs)
    y_true = np.vstack(all_true)

    # ✅ umbral SOLO para SABER/HACER
    y_pred = np.zeros_like(y_true)
    y_pred[:, 0] = (y_prob[:, 0] >= th).astype(int)  # SABER
    y_pred[:, 1] = (y_prob[:, 1] >= th).astype(int)  # HACER

    # ✅ fuerza mínimo 1 de las dos (si ninguna activó)
    for i in range(y_pred.shape[0]):
        if (y_pred[i, 0] + y_pred[i, 1]) == 0:
            j = int(np.argmax(y_prob[i, :2]))  # solo entre saber/hacer
            y_pred[i, j] = 1

    # Métricas SOLO sobre saber/hacer
    y_true2 = y_true[:, :2]
    y_pred2 = y_pred[:, :2]

    return {
        "th": float(th),
        "top_k": int(top_k),
        "f1_micro": float(f1_score(y_true2, y_pred2, average="micro", zero_division=0)),
        "f1_macro": float(f1_score(y_true2, y_pred2, average="macro", zero_division=0)),
        "precision_macro": float(precision_score(y_true2, y_pred2, average="macro", zero_division=0)),
        "recall_macro": float(recall_score(y_true2, y_pred2, average="macro", zero_division=0)),
        "hamming": float(hamming_loss(y_true2, y_pred2)),
        "avg_dims_per_text": float(y_pred2.sum(axis=1).mean()),
        "report": classification_report(y_true2, y_pred2, target_names=DIMENSIONES[:2], zero_division=0)
    }

best = None
for th in np.arange(0.20, 0.61, 0.05):
    m = eval_dim_threshold(float(th), top_k=2)
    print({k: m[k] for k in ["th","f1_micro","f1_macro","hamming","avg_dims_per_text"]})
    if best is None or m["f1_micro"] > best["f1_micro"]:
        best = m

print("\n✅ BEST:", {k: best[k] for k in ["th","f1_micro","f1_macro","hamming","avg_dims_per_text"]})
print("\nReporte calibración SABER/HACER:\n", best["report"])

with open("calibration.json", "w", encoding="utf-8") as f:
    json.dump({
        "dim_threshold": best["th"],
        "dim_top_k": 2,
        "areas": AREAS,
        "dims": DIMENSIONES
    }, f, ensure_ascii=False, indent=2)

print("✅ Exportado: calibration.json")


{'th': 0.2, 'f1_micro': 0.6508474576271186, 'f1_macro': 0.6347833164730643, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 2.0}
{'th': 0.25, 'f1_micro': 0.6508474576271186, 'f1_macro': 0.6347833164730643, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 2.0}
{'th': 0.3, 'f1_micro': 0.6508474576271186, 'f1_macro': 0.6347833164730643, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 2.0}
{'th': 0.35, 'f1_micro': 0.6508474576271186, 'f1_macro': 0.6347833164730643, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 2.0}
{'th': 0.39999999999999997, 'f1_micro': 0.6508474576271186, 'f1_macro': 0.6347833164730643, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 2.0}
{'th': 0.44999999999999996, 'f1_micro': 0.6411149825783972, 'f1_macro': 0.6231884057971014, 'hamming': 0.5175879396984925, 'avg_dims_per_text': 1.9195979833602905}
{'th': 0.49999999999999994, 'f1_micro': 0.5818181818181818, 'f1_macro': 0.5749077336891928, 'hamming': 0.4623115577889447, 'avg_dims_per_text': 1.2462311

In [30]:
# ✅ CELDA 10: Inferencia FINAL (robusta)
# - Área: reglas (porque en tus PDFs hay encabezados como "FÍSICA", "CIENCIAS SOCIALES", etc.)
# - Dim: LSTM
# - SER/DECIDIR: solo si keywords o prob muy alta (gating)

import json
import numpy as np

with open("calibration.json", "r", encoding="utf-8") as f:
    calib = json.load(f)

DIM_TH = float(calib["dim_threshold"])
# MAX_DIMS_OUT = int(calib["dim_top_k"])
MAX_DIMS_OUT = 4


# thresholds más estrictos para raras
TH_SER = 0.65
TH_DECIDIR = 0.75

SER_KWS = ["valorar","respeto","convivir","solidaridad","ético","ética","actitud","equidad","dignidad","inclusión","responsable"]
DEC_KWS = ["decidir","elegir","proponer","planificar","priorizar","comprometer","promover","argumentar","evaluar","seleccionar"]

def has_kw(text, kws):
    t = text.lower()
    return any(k in t for k in kws)

def rule_area(text: str):
    t = text.upper()
    # 1) prioridad por encabezados del PDF
    for a in AREAS:
        if a.replace("_", " ") in t:
            return a
    # 2) fallback a tus reglas existentes (CELDA 6)
    areas = infer_areas(text)
    return areas[0] if areas else "CIENCIAS_SOCIALES"

def pick_dims(text, prob_d):
    # prob_d en orden DIMENSIONES = ["SABER","HACER","SER","DECIDIR"]
    out = []

    # SABER/HACER con threshold calibrado
    if prob_d[0] >= DIM_TH: out.append(0)
    if prob_d[1] >= DIM_TH: out.append(1)

    # SER/DECIDIR con gating (keywords o confianza muy alta)
    if has_kw(text, SER_KWS) or prob_d[2] >= TH_SER:
        out.append(2)
    if has_kw(text, DEC_KWS) or prob_d[3] >= TH_DECIDIR:
        out.append(3)

    if len(out) == 0:
        out = [int(np.argmax(prob_d[:2]))]  # mínimo saber/hacer

    # máximo K dims
    out = sorted(set(out), key=lambda i: prob_d[i], reverse=True)[:MAX_DIMS_OUT]
    return out

def predict_final(text: str):
    model.eval()

    area = rule_area(text)

    enc = tokenizer(text, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
    ids = enc["input_ids"].to(DEVICE)
    att = enc["attention_mask"].to(DEVICE)

    with torch.no_grad():
        _, logits_d = model(ids, att)
        prob_d = torch.sigmoid(logits_d).cpu().numpy()[0]  # [4]

    dims_idx = pick_dims(text, prob_d)
    dims = [DIMENSIONES[i] for i in dims_idx]

    labels = [{"label": f"{area}__{d}", "score": float(prob_d[DIMENSIONES.index(d)])} for d in dims]
    return {"area": area, "labels": labels}

texto = input("Pega aquí un texto:\n")
print(json.dumps(predict_final(texto), ensure_ascii=False, indent=2))


Pega aquí un texto:
Para tu prueba de modelo, he redactado un texto educativo sobre el Cuidado de la Madre Tierra y la Biodiversidad en Bolivia, integrando las dimensiones de la Ser y el Decidir.  En el modelo educativo boliviano, estas dimensiones son fundamentales: el Ser se enfoca en los valores y la identidad, mientras que el Decidir se orienta a la capacidad de actuar con conciencia crítica y transformar la realidad.  🍃 La Biodiversidad y nuestra Responsabilidad con la Madre Tierra Bolivia es uno de los países con mayor biodiversidad en el mundo, albergando desde las cumbres nevadas de los Andes hasta las llanuras amazónicas. Sin embargo, proteger esta riqueza no solo requiere conocimientos científicos, sino una transformación profunda en nuestra forma de convivir con el entorno.  La Dimensión del Ser: Valores y Conciencia En el desarrollo de esta unidad, fortalecemos el Ser mediante la recuperación de valores sociocomunitarios como la complementariedad y el respeto hacia todos lo

In [35]:
# ✅ CELDA 11: Exporta modelo + labels + tokenizer + config (MULTI-HEAD REAL)

EXPORT_DIR = "/content/model_export_red_1_LSTM"
os.makedirs(EXPORT_DIR, exist_ok=True)

model_path = os.path.join(EXPORT_DIR, "pedagogical_lstm.pt")
labels_path = os.path.join(EXPORT_DIR, "labels.json")
tok_path   = os.path.join(EXPORT_DIR, "tokenizer_name.txt")
cfg_path   = os.path.join(EXPORT_DIR, "config.json")

# pesos
torch.save(model.state_dict(), model_path)

# labels (combinaciones AREA__DIM)
with open(labels_path, "w", encoding="utf-8") as f:
    json.dump(LABELS, f, ensure_ascii=False, indent=2)

# tokenizer
with open(tok_path, "w", encoding="utf-8") as f:
    f.write(TOKENIZER_NAME)

# config (usa tu mejor threshold real)
cfg = {
    "model_type": "lstm_multihead",
    "vocab_size": int(tokenizer.vocab_size),
    "pad_token_id": int(tokenizer.pad_token_id),
    "emb_dim": int(EMB_DIM),
    "hidden_dim": int(HIDDEN),
    "max_len": int(MAX_LEN),

    # clases
    "areas": AREAS,
    "dims": DIMENSIONES,
    "labels": LABELS,

    # calibración (tu BEST)
    "threshold_default": 0.20,
    "top_areas": 3,
    "top_k_combos": 8,
    "max_active_combos": 12
}

with open(cfg_path, "w", encoding="utf-8") as f:
    json.dump(cfg, f, ensure_ascii=False, indent=2)

print("✅ Export listo en:", EXPORT_DIR)
print("Archivos:", os.listdir(EXPORT_DIR))


✅ Export listo en: /content/model_export_red_1_LSTM
Archivos: ['labels.json', 'pedagogical_lstm.pt', 'config.json', 'tokenizer_name.txt']


In [39]:
# ✅ CELDA 12: Loader + inferencia FINAL (MULTI-HEAD, igual a tu entrenamiento)

import os, json
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer

# ---------- Modelo MULTI-HEAD ----------
class PedagogicalLSTMMultiHead(nn.Module):
    def __init__(self, vocab_size, pad_token_id, emb_dim, hidden_dim, num_areas, num_dims):
        super().__init__()
        # IMPORTANTE: nombre "embedding" para que coincida con tu state_dict
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_token_id)
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.2)

        # IMPORTANTE: nombres fc_area y fc_dim para que coincidan con tu state_dict
        self.fc_area = nn.Linear(hidden_dim * 2, num_areas)
        self.fc_dim  = nn.Linear(hidden_dim * 2, num_dims)

    def forward(self, input_ids, attention_mask):
        x = self.embedding(input_ids)          # [B,T,E]
        out, _ = self.lstm(x)                  # [B,T,2H]

        mask = attention_mask.unsqueeze(-1).float()
        out = out * mask
        pooled = out.sum(dim=1) / mask.sum(dim=1).clamp(min=1.0)  # masked mean
        pooled = self.dropout(pooled)

        logits_area = self.fc_area(pooled)     # [B,num_areas]
        logits_dim  = self.fc_dim(pooled)      # [B,num_dims]
        return logits_area, logits_dim

# ---------- Loader ----------
def load_classifier(export_dir: str, device: str = "cpu"):
    cfg = json.load(open(os.path.join(export_dir, "config.json"), encoding="utf-8"))
    tok_name = open(os.path.join(export_dir, "tokenizer_name.txt"), encoding="utf-8").read().strip()

    tokenizer = AutoTokenizer.from_pretrained(tok_name)

    areas = cfg["areas"]
    dims  = cfg["dims"]

    model = PedagogicalLSTMMultiHead(
        vocab_size=cfg["vocab_size"],
        pad_token_id=cfg["pad_token_id"],
        emb_dim=cfg["emb_dim"],
        hidden_dim=cfg["hidden_dim"],
        num_areas=len(areas),
        num_dims=len(dims)
    )

    sd = torch.load(os.path.join(export_dir, "pedagogical_lstm.pt"), map_location=device)
    model.load_state_dict(sd, strict=True)
    model.to(device)
    model.eval()

    return model, tokenizer, cfg

# ---------- Utilidades ----------
def build_combo_scores(area_probs, dim_probs, areas, dims):
    """
    Construye scores para todas las combinaciones AREA__DIM como producto:
    score(area__dim) = area_prob * dim_prob
    (estable y consistente para pasar a Red 2)
    """
    combos = []
    for ai, a in enumerate(areas):
        for di, d in enumerate(dims):
            combos.append((f"{a}__{d}", float(area_probs[ai] * dim_probs[di])))
    combos.sort(key=lambda x: -x[1])
    return combos

# ---------- Inferencia FINAL ----------
def classify_text(model, tokenizer, cfg, text: str, threshold: float | None = None, device: str = "cpu"):
    thr = cfg["threshold_default"] if threshold is None else float(threshold)

    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=cfg["max_len"],
        return_tensors="pt"
    )
    ids = enc["input_ids"].to(device)
    att = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits_area, logits_dim = model(ids, att)
        #area_probs = torch.sigmoid(logits_area).cpu().numpy()[0]
        area_probs = torch.softmax(logits_area, dim=-1).cpu().numpy()[0]


        dim_probs  = torch.sigmoid(logits_dim).cpu().numpy()[0]

    areas = cfg["areas"]
    dims  = cfg["dims"]

    # TOP 3 áreas
    area_rank = sorted(
        [{"area": areas[i], "score": float(area_probs[i])} for i in range(len(areas))],
        key=lambda x: -x["score"]
    )
    areas_top3 = area_rank[:cfg["top_areas"]]

    # dims probs (4)
    dims_out = {dims[i]: float(dim_probs[i]) for i in range(len(dims))}

    # combinaciones (AREA__DIM) como producto
    combo_scores = build_combo_scores(area_probs, dim_probs, areas, dims)

    # TOP combos
    top_combos = [{"label": c[0], "score": c[1]} for c in combo_scores[:cfg["top_k_combos"]]]

    # combos activas por threshold (limitadas)
    active = [{"label": c[0], "score": c[1]} for c in combo_scores if c[1] >= thr]
    if not active:
        active = [{"label": combo_scores[0][0], "score": combo_scores[0][1]}]
    active = active[:cfg["max_active_combos"]]

    return {
        "threshold": thr,
        "areas_top": areas_top3,
        "dims_probs": dims_out,
        "combo_top": top_combos,
        "active": active
    }

# ---------- Mini test ----------
export_dir = "/content/model_export_red_1_LSTM"
device = "cuda" if torch.cuda.is_available() else "cpu"

clf_model, clf_tok, clf_cfg = load_classifier(export_dir, device=device)
print("✅ Loader OK | areas:", len(clf_cfg["areas"]), "| dims:", len(clf_cfg["dims"]))

test_text = "Movimiento parabólico, aplicar fórmulas y explicar el procedimiento."
out = classify_text(clf_model, clf_tok, clf_cfg, test_text, device=device)
print(json.dumps(out, ensure_ascii=False, indent=2))


✅ Loader OK | areas: 8 | dims: 4
{
  "threshold": 0.2,
  "areas_top": [
    {
      "area": "FISICA",
      "score": 0.8475247621536255
    },
    {
      "area": "MATEMATICA",
      "score": 0.09015604853630066
    },
    {
      "area": "CIENCIAS_NATURALES_BIOLOGIA",
      "score": 0.034537892788648605
    }
  ],
  "dims_probs": {
    "SABER": 0.5325000882148743,
    "HACER": 0.5034955739974976,
    "SER": 0.4422486126422882,
    "DECIDIR": 0.2825419306755066
  },
  "combo_top": [
    {
      "label": "FISICA__SABER",
      "score": 0.4513069987297058
    },
    {
      "label": "FISICA__HACER",
      "score": 0.42672497034072876
    },
    {
      "label": "FISICA__SER",
      "score": 0.3748166561126709
    },
    {
      "label": "FISICA__DECIDIR",
      "score": 0.239461287856102
    },
    {
      "label": "MATEMATICA__SABER",
      "score": 0.04800810292363167
    },
    {
      "label": "MATEMATICA__HACER",
      "score": 0.04539317265152931
    },
    {
      "label": "MATEMA